# Testing Aer Simulator Methods

This notebook tests different Aer simulator methods with the Qonscious framework.

Available Aer simulation methods:
- **automatic**: Automatically select the best method
- **statevector**: State vector simulation
- **density_matrix**: Density matrix simulation
- **stabilizer**: Stabilizer simulation (Clifford circuits only)
- **matrix_product_state**: Matrix product state simulation
- **extended_stabilizer**: Extended stabilizer simulation
- **unitary**: Unitary matrix simulation
- **superop**: Superoperator simulation

## Setup and Imports

In [ ]:
import json
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit_aer.primitives import SamplerV2 as Sampler
from qonscious.adapters import AerSimulatorAdapter
from qonscious.checks import MeritComplianceCheck
from qonscious.foms import AggregateQPUFigureOfMerit
from qonscious.actions import QonsciousCallable
from qonscious import run_conditionally

## Helper Functions

In [ ]:
def create_test_circuit(n_qubits=3):
    """Create a simple Bell-like test circuit."""
    qc = QuantumCircuit(n_qubits)
    qc.h(0)
    for i in range(n_qubits - 1):
        qc.cx(i, i + 1)
    qc.measure_all()
    return qc

def on_pass(backend_adapter, figureOfMeritResults):
    """Executed if merit check passes."""
    firstFoMResult = figureOfMeritResults[0]
    print("\n--- Checks PASSED ---")
    properties = firstFoMResult.get('properties', {})
    print("\n--- Calculated Properties ---")
    pretty_properties = json.dumps(properties, indent=2)
    print(pretty_properties)
    num_qubits = properties.get('n_qubits', 'N/A')
    print(f"\nNumber of Qubits: {num_qubits}")
    print("-" * 25)
    return None

def on_fail(backend_adapter, figureOfMeritResults):
    """Executed if merit check fails."""
    firstFoMResult = figureOfMeritResults[0]
    print("\n--- Checks FAILED ---")
    properties = firstFoMResult.get('properties', {})
    print("\n--- Calculated Properties ---")
    pretty_properties = json.dumps(properties, indent=2)
    print(pretty_properties)
    return None

## Test Function for Each Method

In [ ]:
def test_aer_method(method_name, n_qubits=3, qubit_threshold=2):
    """
    Test a specific Aer simulator method.
    
    Args:
        method_name: Aer simulation method
        n_qubits: Number of qubits for the simulator
        qubit_threshold: Minimum qubits required to pass check
    """
    print(f"\n{'='*60}")
    print(f"Testing Aer Simulator Method: {method_name}")
    print(f"{'='*60}")
    
    try:
        # Create simulator with specific method
        simulator = AerSimulator(method=method_name)
        sampler = Sampler(backend=simulator)
        
        # Create mock qubit properties (for local simulator)
        qubits_properties = [
            {"T1": [100e-6], "T2": [80e-6]} for _ in range(n_qubits)
        ]
        
        # Create adapter
        adapter = AerSimulatorAdapter(sampler, simulator, qubits_properties)
        
        print(f"\nSimulator Method: {method_name}")
        print(f"Available Qubits: {adapter.n_qubits}")
        
        # Create merit compliance check
        check_QPU = MeritComplianceCheck(
            figure_of_merit=AggregateQPUFigureOfMerit(),
            decision_function=lambda r: r is not None and r["properties"]["n_qubits"] > qubit_threshold
        )
        
        # Run conditional check
        qonscious_result = run_conditionally(
            backend_adapter=adapter,
            checks=[check_QPU],
            on_pass=QonsciousCallable(on_pass),
            on_fail=QonsciousCallable(on_fail)
        )
        
        # Run a test circuit
        print(f"\n--- Running Test Circuit ---")
        circuit = create_test_circuit(min(n_qubits, 3))
        result = adapter.run(circuit, shots=1000)
        print(f"Circuit executed successfully")
        print(f"Result counts: {result['counts']}")
        
        return qonscious_result
        
    except Exception as e:
        print(f"\n❌ Error testing method '{method_name}': {str(e)}")
        return None

## Test 1: Automatic Method

In [ ]:
test_aer_method('automatic', n_qubits=5, qubit_threshold=2)

## Test 2: Statevector Method

In [ ]:
test_aer_method('statevector', n_qubits=5, qubit_threshold=2)

## Test 3: Density Matrix Method

In [ ]:
test_aer_method('density_matrix', n_qubits=5, qubit_threshold=2)

## Test 4: Stabilizer Method

Note: Stabilizer simulation only works with Clifford circuits (H, S, CNOT, etc.)

In [ ]:
test_aer_method('stabilizer', n_qubits=5, qubit_threshold=2)

## Test 5: Matrix Product State Method

In [ ]:
test_aer_method('matrix_product_state', n_qubits=5, qubit_threshold=2)

## Test 6: Extended Stabilizer Method

In [ ]:
test_aer_method('extended_stabilizer', n_qubits=5, qubit_threshold=2)

## Test 7: Unitary Method

Note: Unitary simulation doesn't support measurement, so this may fail

In [ ]:
# Unitary method requires circuits without measurements
# This will likely fail with the current test circuit
test_aer_method('unitary', n_qubits=5, qubit_threshold=2)

## Test 8: Superoperator Method

Note: Superoperator simulation doesn't support measurement, so this may fail

In [ ]:
# Superop method requires circuits without measurements
# This will likely fail with the current test circuit
test_aer_method('superop', n_qubits=5, qubit_threshold=2)

## Test All Methods at Once

In [ ]:
methods = [
    'automatic',
    'statevector',
    'density_matrix',
    'stabilizer',
    'matrix_product_state',
    'extended_stabilizer',
    'unitary',
    'superop'
]

print("\n" + "="*60)
print("TESTING ALL AER SIMULATOR METHODS")
print("="*60)

results = {}
for method in methods:
    result = test_aer_method(method, n_qubits=5, qubit_threshold=2)
    results[method] = result
    print("\n")

print("\n" + "="*60)
print("SUMMARY")
print("="*60)
for method, result in results.items():
    status = "✅ Success" if result is not None else "❌ Failed"
    print(f"{method:25s} - {status}")

## Comparison: Different Qubit Counts

Test how different methods handle varying qubit counts

In [ ]:
# Test with different qubit counts
qubit_counts = [3, 5, 10]
test_methods = ['automatic', 'statevector', 'matrix_product_state']

for n_qubits in qubit_counts:
    print(f"\n{'='*60}")
    print(f"Testing with {n_qubits} qubits")
    print(f"{'='*60}")
    
    for method in test_methods:
        test_aer_method(method, n_qubits=n_qubits, qubit_threshold=2)
        print()